In [8]:
#James Chartouni

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import scipy

from helpers import *

# Import the 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD, FastICA
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler

In [2]:
train = pd.read_csv("data/train_cleaned.csv")
train_orig = pd.read_csv("data/train.csv")
#y_train = train_orig['target'] #TRY A LOG TRANSFORM
y = np.log1p(train_orig["target"].values)
#y_train = boxcox(train_orig["target"].values)[0]
#X_train = train.drop(["target", "ID"], axis=1)
X = train

print ("Rows: " + str(train.shape[0]) + ", Columns: " + str(train.shape[1]))
print(train.head())

test = pd.read_csv('data/test_cleaned.csv')
#test_X = test.drop(["ID"], axis=1)
print ("Rows: " + str(test.shape[0]) + ", Columns: " + str(test.shape[1]))
test.head()

Rows: 4459, Columns: 1420
   SumValues  SumZeros  20aa07010  dc5a8f1d8  4681de4fd  adf119b9a  0d866c3d7  \
0        103      4628   0.000000        0.0        0.0        0.0        0.0   
1         68      4663  14.603968        0.0        0.0        0.0        0.0   
2         19      4712   0.000000        0.0        0.0        0.0        0.0   
3         23      4708   0.000000        0.0        0.0        0.0        0.0   
4         27      4704  14.508658        0.0        0.0        0.0        0.0   

   bd8f989f1  22ed6dba3  92b13ebba       ...             Std  Q4.95  \
0        0.0        0.0        0.0       ...        2.169096    0.0   
1        0.0        0.0        0.0       ...        1.809385    0.0   
2        0.0        0.0        0.0       ...        0.909701    0.0   
3        0.0        0.0        0.0       ...        0.887781    0.0   
4        0.0        0.0        0.0       ...        1.098374    0.0   

          Q5       Skew       mad        kurt       sem  num

,SumValues,SumZeros,20aa07010,dc5a8f1d8,4681de4fd,adf119b9a,0d866c3d7,bd8f989f1,22ed6dba3,92b13ebba,...,Std,Q4.95,Q5,Skew,mad,kurt,sem,number_of_different,non_zero_count,geometric_mean
0,73,4658,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,1.829730,0.0,20.682969,8.348649,0.438792,69.445898,0.026605,83,85,14.457006
1,11,4720,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.645842,0.0,17.273869,22.865799,0.058110,534.362437,0.009391,21,23,9.579583
2,100,4631,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,2.037257,0.0,18.197537,6.921548,0.576216,46.797912,0.029622,84,112,14.151046
3,121,4610,0.0,0.0,0.0,0.0,0.0,16.854494,0.0,0.0,...,2.510632,0.0,18.420681,6.121881,0.785246,35.733973,0.036505,111,133,16.031065
4,9,4722,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.617740,0.0,18.494941,24.977718,0.050213,631.998291,0.008982,19,21,9.390606


In [4]:
X.info()
test.info()
y

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 1420 entries, SumValues to geometric_mean
dtypes: float64(1416), int64(4)
memory usage: 48.3 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 1420 entries, SumValues to geometric_mean
dtypes: float64(1416), int64(4)
memory usage: 534.6 MB


array([ 17.45309674,  13.3046866 ,  16.11809575, ...,  14.84513033,
        16.11809575,  16.81124288])

In [5]:
#### Check if there are any NULL values in Train Data
print("Total Train Features with NaN Values = " + str(X.columns[X.isnull().sum() != 0].size))
if (X.columns[X.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(X.columns[X.isnull().sum() != 0])))
    X[X.columns[X.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total Train Features with NaN Values = 0


In [36]:
def lgbm_auto(data, y_train, test_data):
      
    train_X, val_X, train_y, val_y = train_test_split(data, y_train, test_size = 0.2, random_state = 42)
    
    params = {
        "objective" : "regression",
        "metric" : 'rmse',
        "num_leaves" : 50,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42,
        'colsample_bytree': 0.48018395169871853,
        'max_bin': 349,
        'n_estimators': 194,
        "n_threads": -1,
        'reg_alpha': 1.3428918202554421e-06,
        'reg_lambda': 0.36738052550567774,
        'scale_pos_weight': 0.0077999778087354376,
        'subsample': 0.72080408790270512,
        'subsample_for_bin': 413393,
        'subsample_freq': 8
    }
    
    '''
    params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'rmse',
    "learning_rate": 0.01,
    "num_leaves": 200,
    "feature_fraction": 0.50,
    "bagging_fraction": 0.50,
    'bagging_freq': 4,
    "max_depth": -1,
    "reg_alpha": 0.3,
    "reg_lambda": 0.1,
    #"min_split_gain":0.2,
    "min_child_weight":10,
    'zero_as_missing':True
    }
    
    
    '''
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval],
                      early_stopping_rounds=100, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_data, num_iteration=model.best_iteration))
    #pred_test_y = inv_boxcox(model.predict(test_data, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [ ]:
#normalization 
#minMax_train = MinMaxScaler().fit_transform(X_train.values)
#standardScaler_train = StandardScaler().fit_transform(X_train.values)
#robustScaler_train = RobustScaler().fit_transform(X_train.values)
maxAbsScaler_train = MaxAbsScaler().fit_transform(X.values)


#minMax_test = MinMaxScaler().fit_transform(test_X.values)
#standardScaler_test = StandardScaler().fit_transform(test_X.values)
#robustScaler_test = RobustScaler().fit_transform(test_X.values)
maxAbsScaler_test = MaxAbsScaler().fit_transform(test.values)



In [ ]:
n_components = 20


# Convert to sparse matrix
sparse_X = scipy.sparse.csr_matrix(X.values)
sparse_test = scipy.sparse.csr_matrix(test.values)

#try normalizing and standardizing variables before PCA


print("\nStart decomposition process...")

print("PCA")
pca = PCA(n_components=n_components, random_state=42)
pca_results_train = pca.fit_transform(X)
pca_results_test = pca.transform(test)
print(pca.explained_variance_ratio_)

print("SparcePCA")
sparce_pca = SparsePCA(n_components=n_components, random_state=42)
sparce_pca_results_train = sparce_pca.fit_transform(X)
print("SparcePCA test")
sparce_pca_results_test = sparce_pca.transform(test)
print(sparce_pca.explained_variance_ratio_)

print("tSVD")
tsvd = TruncatedSVD(n_components=n_components, random_state=42)
tsvd_results_train = tsvd.fit_transform(X)
tsvd_results_test = tsvd.transform(test)

print("ICA")
ica = FastICA(n_components=n_components, random_state=42)
ica_results_train = ica.fit_transform(X)
ica_results_test = ica.transform(test)

print("GRP")
grp = GaussianRandomProjection(n_components=Nn_components_COMP, eps=0.1, random_state=42)
grp_results_train = grp.fit_transform(X)
grp_results_test = grp.transform(test)

print("SRP")
srp = SparseRandomProjection(n_components=n_components, dense_output=True, random_state=42)
srp_results_train = srp.fit_transform(X)
srp_results_test = srp.transform(test)

print("TSNE")
tsne = TSNE(n_components=n_components, verbose=1, random_state=42, n_iter=500)
tsne_results_train = tsne.fit_transform(X)
tsne_results_test = tsne.transform(test)

#Factor Analysis

#Kernel PCA

X_train = pd.Dataframe()
X_test = pd.Dataframe()

print("Append decomposition components to datasets...")
for i in range(1, n_components + 1):
    X_train['pca_' + str(i)] = pca_results_train[:, i - 1]
    X_test['pca_' + str(i)] = pca_results_test[:, i - 1]

    X_train['spca_' + str(i)] = sparce_pca_results_train[:, i - 1]
    X_test['spca_' + str(i)] = sparce_pca_results_test[:, i - 1]
    
    X_train['ica_' + str(i)] = ica_results_train[:, i - 1]
    X_test['ica_' + str(i)] = ica_results_test[:, i - 1]

    X_train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    X_test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    X_train['grp_' + str(i)] = grp_results_train[:, i - 1]
    X_test['grp_' + str(i)] = grp_results_test[:, i - 1]

    X_train['srp_' + str(i)] = srp_results_train[:, i - 1]
    X_test['srp_' + str(i)] = srp_results_test[:, i - 1]
    
    X_train['tsne_' + str(i)] = tsne_results_train[:, i - 1]
    X_test['tsne_' + str(i)] = tsne_results_test[:, i - 1]
    

print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))


Start decomposition process...
PCA
[  6.84190249e-01   2.70950982e-01   9.09071526e-03   2.73655643e-03
   1.25786486e-03   7.72638444e-04   6.77609470e-04   6.31381632e-04
   4.96527570e-04   4.56989667e-04   4.34568787e-04   4.03121171e-04
   3.55032209e-04   3.18168560e-04   3.13632850e-04   2.66096493e-04
   2.45900721e-04   2.35228760e-04   2.22150305e-04   2.07768187e-04]
SparcePCA


In [ ]:
#baseline
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(X, y, test)

In [ ]:
#pca
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(pca_results_train, y, pca_results_test)

In [ ]:
#sparce pca
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(sparce_pca_results_train, y, sparce_pca_results_test)

In [ ]:
#tvsd
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(tsvd_results_train, y, tsvd_results_test)

In [ ]:
#ica
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(ica_results_train, y, ica_results_test)

In [ ]:
#grp
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(grp_results_train, y, grp_results_test)

In [ ]:
#srp
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(srp_results_train, y, srp_results_test)

In [ ]:
#tsne
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(tsne_results_train, y, tsne_results_test)

In [ ]:
#combined features 
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(X_train, y, X_test)

In [53]:
COMPONENTS = 20

# List of decomposition methods to use
methods = [
    #TruncatedSVD(n_components=COMPONENTS),
    #PCA(n_components=COMPONENTS),
    #FastICA(n_components=COMPONENTS),
    #GaussianRandomProjection(n_components=COMPONENTS, eps=0.1),
    SparseRandomProjection(n_components=COMPONENTS, dense_output=True)    
]

# Run all the methods
embeddings = []
for method in methods:
    name = method.__class__.__name__    
    embeddings.append(
        pd.DataFrame(method.fit_transform(X), columns=[f"{name}_{i}" for i in range(COMPONENTS)])
    )
    print(f">> Ran {name}")
    
# Put all components into one dataframe
components_df = pd.concat(embeddings, axis=1)

>> Ran SparseRandomProjection


In [54]:
print(components_df.columns)
#print(components_df.info())

Index(['SparseRandomProjection_0', 'SparseRandomProjection_1',
       'SparseRandomProjection_2', 'SparseRandomProjection_3',
       'SparseRandomProjection_4', 'SparseRandomProjection_5',
       'SparseRandomProjection_6', 'SparseRandomProjection_7',
       'SparseRandomProjection_8', 'SparseRandomProjection_9',
       'SparseRandomProjection_10', 'SparseRandomProjection_11',
       'SparseRandomProjection_12', 'SparseRandomProjection_13',
       'SparseRandomProjection_14', 'SparseRandomProjection_15',
       'SparseRandomProjection_16', 'SparseRandomProjection_17',
       'SparseRandomProjection_18', 'SparseRandomProjection_19',
       'SparseRandomProjection_20', 'SparseRandomProjection_21',
       'SparseRandomProjection_22', 'SparseRandomProjection_23',
       'SparseRandomProjection_24', 'SparseRandomProjection_25',
       'SparseRandomProjection_26', 'SparseRandomProjection_27',
       'SparseRandomProjection_28', 'SparseRandomProjection_29',
       'SparseRandomProjection_30',

In [55]:
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(X, y, test)

Training until validation scores don't improve for 100 rounds.
[50]	training's rmse: 1.48089	valid_1's rmse: 1.50144
[100]	training's rmse: 1.31385	valid_1's rmse: 1.41479
[150]	training's rmse: 1.20481	valid_1's rmse: 1.37627
Did not meet early stopping. Best iteration is:
[194]	training's rmse: 1.13459	valid_1's rmse: 1.3614


In [56]:
pred_test_lgbm, model_lgbm, evals_result_lgbm = lgbm_auto(components_df, y, test)

Training until validation scores don't improve for 100 rounds.
[50]	training's rmse: 1.56835	valid_1's rmse: 1.6161
[100]	training's rmse: 1.41656	valid_1's rmse: 1.5645
[150]	training's rmse: 1.29314	valid_1's rmse: 1.53023
Did not meet early stopping. Best iteration is:
[194]	training's rmse: 1.20154	valid_1's rmse: 1.51132


In [57]:
# feature importance
print("Features Importance model_maxAbsScaler...")
gain = model_lgbm.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model_lgbm.feature_name(), 
                   'split':model_lgbm.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:50])

Features Importance model_maxAbsScaler...
                      feature      gain  split
57  SparseRandomProjection_57  4.329577    230
34  SparseRandomProjection_34  3.966076    272
53  SparseRandomProjection_53  3.286816    238
6    SparseRandomProjection_6  2.714925    184
26  SparseRandomProjection_26  2.440892    180
75  SparseRandomProjection_75  2.310752    184
86  SparseRandomProjection_86  2.177950    185
87  SparseRandomProjection_87  1.850851    131
67  SparseRandomProjection_67  1.821966    160
66  SparseRandomProjection_66  1.805911    166
1    SparseRandomProjection_1  1.667898    129
22  SparseRandomProjection_22  1.661434    159
73  SparseRandomProjection_73  1.639419    129
81  SparseRandomProjection_81  1.633417    127
8    SparseRandomProjection_8  1.588384    157
48  SparseRandomProjection_48  1.584912    127
70  SparseRandomProjection_70  1.565727    170
65  SparseRandomProjection_65  1.506796    132
5    SparseRandomProjection_5  1.481498    127
56  SparseRandomPr

In [21]:
#LGBM predictions
sub = pd.read_csv('data/sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test_lgbm

sub["target"] = sub_lgb["target"] 

print(sub.head())
sub.to_csv('submit/sub_lgb.csv', index=False)

          ID        target
0  000137c73  2.048776e+06
1  00021489f  1.976916e+06
2  0004d7953  1.976916e+06
3  00056a333  1.992680e+06
4  00056d8eb  1.976916e+06
